In [ ]:
from PIL import Image
import numpy as np
import glob
import regex 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import learning_curve

In [ ]:
def read_dataset(path):
  data= np.zeros([400,10305])
  i=0
  iamges_list = []
  for filename in glob.glob(path+"/*/*.pgm"): 

    im=Image.open(filename)
    iamges_list.append(im)
    l=regex.split(r'/', filename)
    l2= regex.split(r's', l[1])
    arr = np.array(im)
    data[i,0:10304]=arr.reshape(1,10304)
    data[i, -1]=l2[1]
    i+=1
  return data, iamges_list 

In [ ]:
def data_splitting():
    test_X = np.zeros([200,10305])
    train_X = np.zeros([200,10305])
    test_y= np.zeros(200)
    train_y=np.zeros(200)
    for i in range(data.shape[0]):
        if i%2 == 0:
            test_X[int(i/2)]=data[i,0:10305]
            test_y[int(i/2)]= data[i,10304]
        else:
            train_X[int((i-1)/2)]=data[i,0:10305]
            train_y[int((i-1)/2)]= data[i,10304] 
    return train_X, train_y, test_X, test_y


In [ ]:
def visualize(images,i):
  plt.imshow(images[i])
  

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
def get_classes(data):
    classes= []
    for i in range (41):
        indecies= np.argwhere(data[:,-1]== i)
        classes.append(data[indecies,:].reshape(len(indecies),data.shape[1]))      
    classes.pop(0)
    return classes

def centralized(classes,classes_Mean):
    z=[]
    for i, x in enumerate(classes):
        z.append(x - (np.ones(x.shape)*classes_Mean[i]))
    z=np.array(z)
    #Z=Z.reshape(400,10305)
    
    return z

In [ ]:
def LDA(data):
    classes= get_classes(data)
    Means= np.mean(data,0)
    
    list = [np.mean(x, axis=0) for x in classes]
    classes_Mean= np.array(list)   
    print(classes_Mean.shape)
    
    SB=0
    for i, x in enumerate(classes):
        SB+=np.dot(x.shape[0]*(classes_Mean[i] - Means),(classes_Mean[1] - Means).T)

    ##..........center class matrices..........##
    Z= certerlaized(classes,classes_Mean) 
    
    ##..........Within class scatter matrix...........##
    s= []
    S_sum= np.zeros([Z.shape[1], Z.shape[1]])
    for i in range (len(classes)):
        s.append(Z[i].dot(Z[i].T))
        S_sum+=Z[i].dot(Z[i].T)
    print("number of classes=", len(s), "Summ_of S=", S_sum.shape)
    
    
    eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(S_sum), SB))
    print(eigvec.shape)
    
LDA(data)

In [ ]:
def KNN_classifer(X_train_transformed, y_train,preprocessor_type):

    #................................training........................................#    
    parameters = {'n_neighbors':[1,3,5,7]}
    KNN=KNeighborsClassifier()
    cv= KFold(n_splits=5,shuffle=True, random_state=5)
    KNN_opt = GridSearchCV(KNN, parameters,cv=cv,n_jobs=-1,return_train_score=True)
    KNN_opt.fit(X_train_transformed,  y_train)
    print("Best parameters:",KNN_opt.best_params_)
    print("Best score=",KNN_opt.best_score_ )
    mean_train_score=KNN_opt.cv_results_['mean_train_score']
    mean_test_score=KNN_opt.cv_results_['mean_test_score']
    param= KNN_opt.cv_results_['params']
    
    
    #..........................visualizing learning curves and tuning acuracies.......................#   
    title = "Learning Curves for best estimator using  "+ preprocessor_type
    plt=plot_learning_curve(KNN_opt, title, X_train_transformed,  y_train, cv=cv)
    plt.show()
    
    plt.plot(parameters['n_neighbors'], mean_test_score, 'ro')
    plt.show
    return mean_train_score, mean_test_score, parameters



In [ ]:
data, images= read_dataset("orl_faces") 
train_x, train_y, test_x, test_y= data_splitting()


## call PCA here and get PCA transformed X_train

## first for PCA
#preprocessor_type= "PCA"
#mean_train_score, mean_test_score,parameters =KNN_classifer(train_x, train_y, preprocessor_type)


## call PCA here and get PCA transformed X_train

## second for LDA 
preprocessor_type="LDA"
mean_train_score, mean_test_score,parameters =KNN_classifer(train_x, train_y,preprocessor_type)